In [6]:
import os
import re
import json
import warnings

import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
from tabulate import tabulate
from skopt import BayesSearchCV

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import (
    precision_recall_fscore_support,
    accuracy_score,
    classification_report,
)
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.exceptions import ConvergenceWarning

from utils_taller3 import SenticLexiconFeaturizer
from data.EN_Lexicons.senticnet5 import senticnet

warnings.simplefilter("ignore", UserWarning)

ACTUAL_PATH = os.getcwd()
PATH_20N = os.path.join(ACTUAL_PATH, "data/20news-18828")
PATH_MD = os.path.join(ACTUAL_PATH, "data/Multi Domain Sentiment/processed_acl")
PATH_FINAL_FILES = os.path.join(ACTUAL_PATH, "data/final_files")

stemmer = nltk.stem.SnowballStemmer("english")
nltk.download("stopwords")

RANDOM_STATE = 42
val_ratio_within_train = 1.0 / 7.0

[nltk_data] Downloading package stopwords to /home/erich/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Parte 2

In [ ]:
def evaluate(opt, X_test, y_test, print_flag=False):
    """
    Evalúa un modelo optimizado (BayesSearchCV) en el conjunto de test.
    Devuelve un diccionario con métricas.
    """
    y_pred = opt.predict(X_test)

    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        y_test, y_pred, average="macro"
    )
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
        y_test, y_pred, average="micro"
    )

    resultados = {
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        "precision_micro": precision_micro,
        "recall_micro": recall_micro,
        "f1_micro": f1_micro,
        "accuracy": accuracy_score(y_test, y_pred),
        "best_params": opt.best_params_,
    }
    if print_flag:
        print(classification_report(y_test, y_pred))

    return resultados


def top_features_pos_neg(est, k):
    """
    Funcion que devuelve los mas positivos y los mas negativos
    """
    pipe = est.best_estimator_ if hasattr(est, "best_estimator_") else est
    vect = pipe.named_steps["vect"]
    lr = pipe.named_steps["model"]

    nombres = vect.get_feature_names_out()
    w = lr.coef_.ravel()

    idx_pos = np.argsort(w)[-k:][::-1]
    idx_neg = np.argsort(w)[:k]

    top_pos = list(zip(nombres[idx_pos], w[idx_pos]))
    top_neg = list(zip(nombres[idx_neg], w[idx_neg]))
    return top_pos, top_neg


def mostrar_resultados_tabulate(resultados_segunda_parte, ordenar_por="f1_macro"):
    """
    Muestra los resultados en formato de tabla usando tabulate.
    Ordena por la métrica especificada (default: f1_macro).
    """
    ejemplo = next(iter(resultados_segunda_parte.values()))
    columnas = ["Modelo"] + list(ejemplo.keys())

    filas = []
    for modelo, metricas in resultados_segunda_parte.items():
        fila = [modelo]
        for valor in metricas.values():
            fila.append(round(valor, 4) if isinstance(valor, (int, float)) else valor)
        filas.append(fila)

    if ordenar_por in ejemplo:
        idx = columnas.index(ordenar_por)
        filas.sort(key=lambda x: x[idx], reverse=True)

    print(tabulate(filas, headers=columnas, tablefmt="grid"))

In [ ]:
def parse_review_line(line):
    """
    Devuelve (dict(token->count), label_or_None)
    """
    line = line.strip()
    label = None
    m = re.search(r"#label#:(positive|negative)$", line)
    if m:
        label = m.group(1)
        line = line[: m.start()].strip()
    parts = line.split()
    d = {}
    for p in parts:
        if ":" in p:
            tok, cnt = p.rsplit(":", 1)
            try:
                d[tok] = int(cnt)
            except:
                try:
                    d[tok] = float(cnt)
                except:
                    d[tok] = 1
    return d, label


def load_reviews(filepath):
    X = []
    y = []
    with open(filepath, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            d, label = parse_review_line(line)
            if d:
                X.append(d)
                y.append(label)
    return X, y


def build_train_test(base_path: str):
    domains = [
        d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))
    ]
    data = {}

    for domain in domains:
        domain_path = os.path.join(base_path, domain)

        X_pos, y_pos = load_reviews(os.path.join(domain_path, "positive.review"))
        X_neg, y_neg = load_reviews(os.path.join(domain_path, "negative.review"))
        X_unl, y_unl = load_reviews(os.path.join(domain_path, "unlabeled.review"))

        min_len = min(len(X_pos), len(X_neg))
        # print(max(len(X_pos), len(X_neg)))
        X_pos, y_pos = X_pos[:min_len], y_pos[:min_len]
        X_neg, y_neg = X_neg[:min_len], y_neg[:min_len]

        data[domain] = {"X": X_pos + X_neg, "y": y_pos + y_neg, "rest": (X_unl, y_unl)}

        print(
            f"[{domain}] Positive: {len(X_pos)}, Negative: {len(X_neg)}, Rest(unlabeled): {len(X_unl)}"
        )

    return data


def generar_modelo_val_train(data_domain, espacio, model, mode="bow", iteraciones=30):
    """
    Entrena y valida un modelo con BayesSearchCV para un dominio específico.

    Parametetros
    ----------
    data_domain : dict con estructura {"X":..., "y":..., "rest": (X_unl, y_unl)}
    espacio     : dict con los hiperparámetros a optimizar
    model       : estimador sklearn (e.g. LogisticRegression, MultinomialNB)
    mode        : "bow" o "tfidf"
    iteraciones : número de iteraciones de búsqueda
    """

    X_train, y_train = data_domain["X"], data_domain["y"]
    # X_test, y_test = data_domain["rest"]

    if mode == "bow":
        steps = [("vect", DictVectorizer()), ("model", model)]
    elif mode == "lexicon":
        steps = [("vect", SenticLexiconFeaturizer(senticnet))]
        if isinstance(model, MultinomialNB):
            steps.append(("nonneg", MinMaxScaler()))  # ← asegura X ≥ 0
        steps.append(("model", model))

    elif mode == "tfidf":
        steps = [
            ("vect", DictVectorizer()),
            ("tfidf", TfidfTransformer()),
            ("model", model),
        ]
    else:
        raise ValueError("mode debe ser 'bow' o 'tfidf'")
    pipeline = Pipeline(steps)

    opt = BayesSearchCV(
        estimator=pipeline,
        search_spaces=espacio,
        n_iter=iteraciones,
        cv=10,
        scoring="f1_macro",
        refit=True,
        random_state=RANDOM_STATE,
        n_jobs=-1,
        verbose=0,
    )

    opt.fit(X_train, y_train)
    print("Mejores hiperparámetros:", opt.best_params_)

    # y_pred = opt.predict(X_test)
    # print(classification_report(y_test, y_pred))

    return opt


def entrenar_y_evaluar_por_clase(data, espacios, iteraciones=30):
    """
    Entrena y evalúa 6 modelos por clase:
    - Lexicon + NB
    - Lexicon + LR
    - BoW + NB
    - BoW + LR
    - TF-IDF + NB
    - TF-IDF + LR
    """
    resultados_segunda_parte = {}
    mejores_lr_por_clase = {}  # ← añadimos esto
    modelos_lr_por_clase = {}

    for clase, valores in data.items():
        print(f"\nEntrenando modelos para la clase: {clase}")

        print(f"[{clase}] Entrenando LEXICON + NB")
        opt_lex_nb = generar_modelo_val_train(
            valores,
            espacios["lex_nb"],
            MultinomialNB(),
            mode="lexicon",
            iteraciones=iteraciones,
        )
        resultados_segunda_parte[f"{clase}_LEXICON_NB"] = evaluate(
            opt_lex_nb, valores["rest"][0], valores["rest"][1], print_flag=True
        )

        print(f"[{clase}] Entrenando LEXICON + LR")
        opt_lex_lr = generar_modelo_val_train(
            valores,
            espacios["lex_lr"],
            LogisticRegression(max_iter=1000),
            mode="lexicon",
            iteraciones=iteraciones,
        )
        resultados_segunda_parte[f"{clase}_LEXICON_LR"] = evaluate(
            opt_lex_lr, valores["rest"][0], valores["rest"][1], print_flag=True
        )

        print(f"[{clase}] Entrenando BoW + NB")
        opt_bow_nb = generar_modelo_val_train(
            valores,
            espacios["bow_nb"],
            MultinomialNB(),
            mode="bow",
            iteraciones=iteraciones,
        )
        resultados_segunda_parte[f"{clase}_BoW_NB"] = evaluate(
            opt_bow_nb, valores["rest"][0], valores["rest"][1], print_flag=True
        )

        print(f"[{clase}] Entrenando BoW + LR")
        opt_bow_lr = generar_modelo_val_train(
            valores,
            espacios["bow_lr"],
            LogisticRegression(max_iter=1000),
            mode="bow",
            iteraciones=iteraciones,
        )
        resultados_segunda_parte[f"{clase}_BoW_LR"] = evaluate(
            opt_bow_lr, valores["rest"][0], valores["rest"][1], print_flag=True
        )

        print(f"[{clase}] Entrenando TF-IDF + NB")
        opt_tfidf_nb = generar_modelo_val_train(
            valores,
            espacios["tfidf_nb"],
            MultinomialNB(),
            mode="tfidf",
            iteraciones=iteraciones,
        )
        resultados_segunda_parte[f"{clase}_TFIDF_NB"] = evaluate(
            opt_tfidf_nb, valores["rest"][0], valores["rest"][1], print_flag=True
        )

        print(f"[{clase}] Entrenando TF-IDF + LR")
        opt_tfidf_lr = generar_modelo_val_train(
            valores,
            espacios["tfidf_lr"],
            LogisticRegression(max_iter=1000),
            mode="tfidf",
            iteraciones=iteraciones,
        )
        resultados_segunda_parte[f"{clase}_TFIDF_LR"] = evaluate(
            opt_tfidf_lr, valores["rest"][0], valores["rest"][1], print_flag=True
        )

        modelos_lr_por_clase[clase] = {
            "lexicon": opt_lex_lr,
            "bow": opt_bow_lr,
            "tfidf": opt_tfidf_lr,
        }
        mejor_lr = max(
            [opt_lex_lr, opt_bow_lr, opt_tfidf_lr], key=lambda o: o.best_score_
        )
        mejores_lr_por_clase[clase] = mejor_lr

    return resultados_segunda_parte, mejores_lr_por_clase


espacios = {
    "bow_nb": {
        "model__alpha": (1e-3, 1.0, "log-uniform"),
        "model__fit_prior": [True, False],
    },
    "bow_lr": {
        "model__C": (1e-3, 1e2, "log-uniform"),
        "model__solver": ["liblinear", "lbfgs", "saga"],
        "model__penalty": [
            "l2",
            None,
        ],
        "model__class_weight": [None, "balanced"],
        "model__max_iter": (50, 150),
    },
    "tfidf_nb": {
        "model__alpha": (1e-3, 1.0, "log-uniform"),
        "model__fit_prior": [True, False],
    },
    "tfidf_lr": {
        "model__C": (1e-3, 1e2, "log-uniform"),
        "model__solver": ["liblinear", "lbfgs", "saga"],
        "model__penalty": ["l2", None],
        "model__class_weight": [None, "balanced"],
        "model__max_iter": (50, 150),
    },
    "lex_nb": {
        "model__alpha": (1e-3, 1.0, "log-uniform"),
        "model__fit_prior": [True, False],
    },
    "lex_lr": {
        "model__C": (1e-3, 1e2, "log-uniform"),
        "model__solver": ["liblinear", "lbfgs", "saga"],
        "model__penalty": ["l2", None],
        "model__class_weight": [None, "balanced"],
        "model__max_iter": (50, 150),
    },
}

In [ ]:
path = "./data/Multi Domain Sentiment/processed_acl"
dataset = build_train_test(path)
resultados_segunda_parte, mejores_lr_por_clase = entrenar_y_evaluar_por_clase(
    dataset, espacios, iteraciones=5
)

[electronics] Positive: 1000, Negative: 1000, Rest(unlabeled): 5681
[kitchen] Positive: 1000, Negative: 1000, Rest(unlabeled): 5945
[dvd] Positive: 1000, Negative: 1000, Rest(unlabeled): 3586
[books] Positive: 1000, Negative: 1000, Rest(unlabeled): 4465

Entrenando modelos para la clase: electronics
[electronics] Entrenando LEXICON + NB
Mejores hiperparámetros: OrderedDict([('model__alpha', 0.016994636371262764), ('model__fit_prior', False)])
              precision    recall  f1-score   support

    negative       0.65      0.64      0.65      2824
    positive       0.65      0.66      0.66      2857

    accuracy                           0.65      5681
   macro avg       0.65      0.65      0.65      5681
weighted avg       0.65      0.65      0.65      5681

[electronics] Entrenando LEXICON + LR
Mejores hiperparámetros: OrderedDict([('model__C', 11.533999859559563), ('model__class_weight', None), ('model__max_iter', 110), ('model__penalty', None), ('model__solver', 'lbfgs')])
    

In [11]:
mostrar_resultados_tabulate(resultados_segunda_parte)

+------------------------+-------------------+----------------+------------+-------------------+----------------+------------+------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Modelo                 |   precision_macro |   recall_macro |   f1_macro |   precision_micro |   recall_micro |   f1_micro |   accuracy | best_params                                                                                                                                                             |
+========================+===================+================+============+===================+================+============+============+=========================================================================================================================================================================+
| kitchen_TFIDF_LR       |            0.891  |         0.891  |     0.

In [ ]:
for key in mejores_lr_por_clase.keys():
    print(f"\n Clase :{key}")
    pos, neg = top_features_pos_neg(mejores_lr_por_clase[key], k=5)
    print("\nAtributos mas positivos")
    for key, value in pos:
        print(f"el token {key} con un peso de {value:.2f}")
    print("\nAtributos mas negativos")
    for key, value in neg:
        print(f"el token {key} con un peso de {value:.2f}")


 Clase :electronics

Atributos mas positivos
el token great con un peso de 0.86
el token excellent con un peso de 0.66
el token price con un peso de 0.56
el token perfect con un peso de 0.53
el token best con un peso de 0.45

Atributos mas negativos
el token not con un peso de -0.57
el token poor con un peso de -0.49
el token bad con un peso de -0.43
el token work con un peso de -0.36
el token back con un peso de -0.36

 Clase :kitchen

Atributos mas positivos
el token great con un peso de 0.88
el token easy con un peso de 0.71
el token love con un peso de 0.61
el token best con un peso de 0.55
el token excellent con un peso de 0.54

Atributos mas negativos
el token not con un peso de -0.66
el token disappointed con un peso de -0.58
el token poor con un peso de -0.43
el token too con un peso de -0.39
el token return con un peso de -0.37

 Clase :dvd

Atributos mas positivos
el token great con un peso de 0.58
el token best con un peso de 0.47
el token excellent con un peso de 0.39
el t

In [ ]:
def preparar_dataset_global(data):
    """
    Une todos los subconjuntos de categorías en un dataset único (train+test).
    """
    X_train, y_train, X_test, y_test = [], [], [], []

    for data_domain in data.values():
        X_train.extend(data_domain["X"])
        y_train.extend(data_domain["y"])
        X_test.extend(data_domain["rest"][0])
        y_test.extend(data_domain["rest"][1])

    return {"X": X_train, "y": y_train, "rest": (X_test, y_test)}


def entrenar_y_evaluar_global(data_domain, espacios, iteraciones=30):
    resultados = {}

    print(f"LEXICON + NB")
    opt_lex_nb = generar_modelo_val_train(
        data_domain,
        espacios["lex_nb"],
        MultinomialNB(),
        mode="lexicon",
        iteraciones=iteraciones,
    )
    resultados["LEXICON_NB"] = evaluate(
        opt_lex_nb, data_domain["rest"][0], data_domain["rest"][1], print_flag=True
    )

    print("LEXICON + LR")
    opt_lex_lr = generar_modelo_val_train(
        data_domain,
        espacios["lex_lr"],
        LogisticRegression(),
        mode="lexicon",
        iteraciones=iteraciones,
    )
    resultados["LEXICON_LR"] = evaluate(
        opt_lex_lr, data_domain["rest"][0], data_domain["rest"][1], print_flag=True
    )
    print("BoW_NB")
    opt_bow_nb = generar_modelo_val_train(
        data_domain,
        espacios["bow_nb"],
        MultinomialNB(),
        mode="bow",
        iteraciones=iteraciones,
    )
    resultados["BoW_NB"] = evaluate(opt_bow_nb, *data_domain["rest"], print_flag=True)
    print("BoW_LR")
    opt_bow_lr = generar_modelo_val_train(
        data_domain,
        espacios["bow_lr"],
        LogisticRegression(),
        mode="bow",
        iteraciones=iteraciones,
    )
    resultados["BoW_LR"] = evaluate(opt_bow_lr, *data_domain["rest"], print_flag=True)
    print("TFIDF_NB")
    opt_tfidf_nb = generar_modelo_val_train(
        data_domain,
        espacios["tfidf_nb"],
        MultinomialNB(),
        mode="tfidf",
        iteraciones=iteraciones,
    )
    resultados["TFIDF_NB"] = evaluate(
        opt_tfidf_nb, *data_domain["rest"], print_flag=True
    )
    print("TFIDF_LR")
    opt_tfidf_lr = generar_modelo_val_train(
        data_domain,
        espacios["tfidf_lr"],
        LogisticRegression(),
        mode="tfidf",
        iteraciones=iteraciones,
    )

    mejor_lr = max([opt_lex_lr, opt_bow_lr, opt_tfidf_lr], key=lambda o: o.best_score_)

    return resultados, mejor_lr

In [ ]:
path = "./data/Multi Domain Sentiment/processed_acl"
dataset = build_train_test(path)
dataset_global = preparar_dataset_global(dataset)

resultados_globales, mejor_lr = entrenar_y_evaluar_global(
    dataset_global, espacios, iteraciones=5
)

[electronics] Positive: 1000, Negative: 1000, Rest(unlabeled): 5681
[kitchen] Positive: 1000, Negative: 1000, Rest(unlabeled): 5945
[dvd] Positive: 1000, Negative: 1000, Rest(unlabeled): 3586
[books] Positive: 1000, Negative: 1000, Rest(unlabeled): 4465
LEXICON + NB
Mejores hiperparámetros: OrderedDict([('model__alpha', 0.3252108800594495), ('model__fit_prior', False)])
              precision    recall  f1-score   support

    negative       0.64      0.64      0.64      9795
    positive       0.64      0.65      0.65      9882

    accuracy                           0.64     19677
   macro avg       0.64      0.64      0.64     19677
weighted avg       0.64      0.64      0.64     19677

LEXICON + LR
Mejores hiperparámetros: OrderedDict([('model__C', 11.533999859559563), ('model__class_weight', None), ('model__max_iter', 110), ('model__penalty', None), ('model__solver', 'lbfgs')])
              precision    recall  f1-score   support

    negative       0.66      0.66      0.66     

In [ ]:
mostrar_resultados_tabulate(resultados_globales, ordenar_por="f1_macro")

+----------+-------------------------------------------------------+-------------------+----------------+------------+-------------------+----------------+------------+------------+
| Modelo   | best_params                                           |   precision_macro |   recall_macro |   f1_macro |   precision_micro |   recall_micro |   f1_micro |   accuracy |
+==========+=======================================================+===================+================+============+===================+================+============+============+
| BoW_LR   | OrderedDict([('model__C', 0.11233621690895233)])      |            0.8734 |         0.8734 |     0.8734 |            0.8734 |         0.8734 |     0.8734 |     0.8734 |
+----------+-------------------------------------------------------+-------------------+----------------+------------+-------------------+----------------+------------+------------+
| TFIDF_NB | OrderedDict([('model__alpha', 0.016994636371262764)]) |            0.8518 |  

In [ ]:
print(f"\n Mejores atributos")
pos, neg = top_features_pos_neg(mejor_lr, k=5)
print("\nAtributos mas positivos")
for key, value in pos:
    print(f"el token {key} con un peso de {value:.2f}")
print("\nAtributos mas negativos")
for key, value in neg:
    print(f"el token {key} con un peso de {value:.2f}")


 Mejores atributos

Atributos mas positivos
el token great con un peso de 30.32
el token excellent con un peso de 24.99
el token best con un peso de 20.01
el token perfect con un peso de 17.30
el token easy con un peso de 16.40

Atributos mas negativos
el token not con un peso de -27.21
el token bad con un peso de -20.67
el token disappointed con un peso de -18.68
el token poor con un peso de -18.13
el token disappointing con un peso de -16.92
